In [1]:
from source.denoiser import get_unet

In [2]:
unet = get_unet(input_shape=(32, 32, 3), depths=[4, 4, 4], initial_dim=48)

In [3]:
unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 1, 1)]    0           []                               
                                                                                                  
 Input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 sinusoidal_embedding (Sinusoid  (None, 1, 1, 48)    0           ['input_1[0][0]']                
 alEmbedding)                                                                                     
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 48)   192         ['Input[0][0]']              